In [ ]:
import os
import math
import datetime

In [ ]:
from tqdm import tqdm

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install bert

In [ ]:
!pip install bert-for-tf2

In [ ]:
!pip install bert-tensorflow

In [ ]:
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

ImportError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

## Data

In [ ]:
import json
with open('data_full.json') as json_file:
    CLINC150 = json.load(json_file)
CLINC150_train=CLINC150['train']
CLINC150_test=CLINC150['test']
CLINC150_val=CLINC150['val']

In [ ]:
classes=['insurance',
 'next_holiday',
 'repeat',
 'credit_limit_change',
 'book_hotel',
 'yes',
 'damaged_card',
 'rewards_balance',
 'time',
 'pto_balance',
 'interest_rate',
 'change_volume',
 'taxes',
 'sync_device',
 'traffic',
 'what_song',
 'shopping_list',
 'todo_list_update',
 'order_checks',
 'shopping_list_update']

In [ ]:
train_data=[]
test_data=[]
val_data=[]

In [ ]:
for c in CLINC150_train:
    if c[1] in classes:
        train_data.append(c)

In [ ]:
for c in CLINC150_test:
    if c[1] in classes:
        test_data.append(c)

In [ ]:
for c in CLINC150_val:
    if c[1] in classes:
        val_data.append(c)

In [ ]:
df = pd.DataFrame(train_data)
df.to_csv('train_data.csv', index=False,header=('text','intent'))
train=pd.read_csv('train_data.csv')
print(len(train))
train.head()

In [ ]:
df = pd.DataFrame(val_data)
df.to_csv('val_data.csv', index=False,header=('text','intent'))
valid=pd.read_csv('val_data.csv')
print(len(valid))
valid.head()

In [ ]:
df = pd.DataFrame(test_data)
df.to_csv('test_data.csv', index=False,header=('text','intent'))
test=pd.read_csv('test_data.csv')
print(len(test))
test.head()

In [ ]:
train = train.append(valid).reset_index(drop=True)

In [ ]:
os.makedirs("model", exist_ok=True)
bert_model_name="uncased_L-12_H-768_A-12"
bert_ckpt_dir = os.path.join("model/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

## Input Text Preparation

In [ ]:
class DataPreparation:

    text_column = "text"
    label_column = "intent"

    def __init__(self, train, test, tokenizer: FullTokenizer, classes, max_seq_len=192):
        self.tokenizer = tokenizer
        self.max_seq_len = 0
        self.classes = classes

        ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self.prepare_data, [train, test])

        print("max seq_len", self.max_seq_len)
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        self.train_x, self.test_x = map(self.data_padding, [self.train_x, self.test_x])

    def prepare_data(self, df):
        x, y = [], []

        for _, row in tqdm(df.iterrows()):
            text, label = row[DataPreparation.text_column], row[DataPreparation.label_column]
            tokens = self.tokenizer.tokenize(text)
            tokens = ["[CLS]"] + tokens + ["[SEP]"]
            token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            self.max_seq_len = max(self.max_seq_len, len(token_ids))
            x.append(token_ids)
            y.append(self.classes.index(label))

        return np.array(x), np.array(y)

    def data_padding(self, ids):
        x = []
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
        return np.array(x)

In [ ]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))

## Model

In [ ]:
def model_defination(max_seq_len, bert_ckpt_file):

    with tf.io.gfile.GFile(bert_config_file, "r") as reader:
        bc = StockBertConfig.from_json_string(reader.read())
        bert_params = map_stock_config_to_params(bc)
        bert_params.adapter_size = None
        bert = BertModelLayer.from_params(bert_params, name="bert")

    input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
    bert_output = bert(input_ids)

    print("bert shape", bert_output.shape)

    cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    cls_out = keras.layers.Dropout(0.5)(cls_out)
    logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(units=len(classes), activation="softmax")(logits)

    model = keras.Model(inputs=input_ids, outputs=logits)
    model.build(input_shape=(None, max_seq_len))

    load_stock_weights(bert, bert_ckpt_file)

    return model

In [ ]:
classes = train.intent.unique().tolist()

data = DataPreparation(train, test, tokenizer, classes, max_seq_len=128)

In [ ]:
data.train_x.shape

In [ ]:
data.train_x[0]

In [ ]:
data.train_y[0]

In [ ]:
model = model_defination(data.max_seq_len, bert_ckpt_file)

In [ ]:
model.summary()

In [ ]:
model.compile(
  optimizer=keras.optimizers.Adam(1e-5),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [ ]:
log_dir = "log/intent_detection/" + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[-3]
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

history = model.fit(
  x=data.train_x,
  y=data.train_y,
  validation_split=0.1,
  batch_size=16,
  shuffle=True,
  epochs=5,
  callbacks=[tensorboard_callback]
)

In [ ]:
_, train_acc = model.evaluate(data.train_x, data.train_y)
_, test_acc = model.evaluate(data.test_x, data.test_y)

## Accuracy

In [ ]:
print("train acc", train_acc)
print("test acc", test_acc)

## Let's Try

In [ ]:
sentences = [
  "is it six o clock yet",
  "find me a hotel with good reviews in phoenix"
]
pred_tokens = map(tokenizer.tokenize, sentences)
pred_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], pred_tokens)
pred_token_ids = list(map(tokenizer.convert_tokens_to_ids, pred_tokens))
pred_token_ids = map(
  lambda tids: tids +[0]*(data.max_seq_len-len(tids)),
  pred_token_ids
)
pred_token_ids = np.array(list(pred_token_ids))
predictions = model.predict(pred_token_ids).argmax(axis=-1)
for text, label in zip(sentences, predictions):
    print("text:", text, "\nintent:", classes[label])
    print()